Questo notebook rappresenta un esempio di utilizzo della libreria transformer per interagire con LLM Open Source.<br>
Tutti questi esempi sono tratti dal libro "Hand-on Large Language Models"

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    torch_dtype="auto",
    trust_remote_code="True",
)
tokenizer = AutoTokenizer.from_pretrained( "microsoft/Phi-4-mini-instruct")

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00, 74.54it/s]


## Pipeline

Il modo più semplice è quello di utilizzare una pipeline: in questo caso per text-generation

In [5]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,
)

messages = [
    {"role":     "user",
     "content" : "Create a funny joke about chickens."}
]

output=generator(messages)
print(output[0]["generated_text"])

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Why don't chickens play poker in the jungle? Too many pecking problems!


## Tokenization

Per capire il concetto di tokenizzazione è possibile scendere di livello rispetto alla pipeline ed utilizzare prima il tokenizer per tokenizzare l'input e poi il model per elaborare l'input tokenizzato

In [6]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>Subject: Sincere Apologies for the Gardening Mishap


Dear Sarah,


I hope this message


Avendo spezzato il processo tra tokenizzazione e generazione è possibile verificare come l'input viene trasformato dal tokenizer in un array di interi

In [7]:
input_ids

tensor([[ 10930,    448,   3719,  39950,   6396,    316,  32145,    395,    290,
          62374,  66241,  80785,    403,     13, 115474,   1495,    480,  12570,
             13, 200019]])

Sempre utilizzando il tokenizer è possibile riconvertire questi interi nei rispettivi token

In [8]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
 an
 email
 apolog
izing
 to
 Sarah
 for
 the
 tragic
 gardening
 mish
ap
.
 Explain
 how
 it
 happened
.
<|assistant|>


Anche l'output generato dal modello è costituito da token.<br>
Questi token contengono sia l'input al modello che l'output generato

In [9]:
generation_output

tensor([[ 10930,    448,   3719,  39950,   6396,    316,  32145,    395,    290,
          62374,  66241,  80785,    403,     13, 115474,   1495,    480,  12570,
             13, 200019,  18174,     25,    336,   2768,    512,   6537,  10384,
            395,    290, 193145, 147276,    403,   2499,  36210,  32145, 123200,
             40,   5498,    495,   3176]])

In [10]:
print(tokenizer.decode(3176))

 message


# Confronto tra tokenizers

Test di come un testo venga tokenizzato diversamente a seconda del tokenizer utilizzato

In [11]:
text = """

English and CAPITALIZATION

🎵鸟
show_tokens False None elif == >= else: two tabs:" " Three tabs: "   "

12.0*50=600

"""

In [12]:
colors_list = [
    '102;194;165', '252;141;98', '141;160;203', 
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' + 
            tokenizer.decode(t) + 
            '\x1b[0m', 
            end=' '
        )

Come primo esempio vediamo <b>WordPiece</b> (usato per Bert)

Da notare cone tutte le lettere siano minuscole, di come non vengano considerate le newline e l'esistenza di unknow tokens[UNK]

In [13]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

Vediamo il tokenizer <b>BPE</b> utilizzato da GPT-2

Da notare il carattere newline, la presenza di maiuscole e la tokenizzazione dei caratteri simbolici


In [14]:
show_tokens(text, "gpt2")


 
 English  and  CAP ITAL IZ ATION 
 
 � � � � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"  "  Three  tabs :  "      " 
 
 12 . 0 * 50 = 600 

 

Vediamo ora Flan-T5 che utilizza <b>SentencePiece</b>

Anche in questo caso non viene gestito newline e abbiamo unknown per i token non Inglesi e Emoji

In [15]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

Infine vediamo il tokenization schema di phi4 che utilizza anche lui BPE

In [16]:
show_tokens(text, "microsoft/Phi-4-mini-instruct")



 English  and  CAPITAL IZATION 

 � � 鸟 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"  "  Three  tabs :  "     "

 12 . 0 * 50 = 600 

 

# EMBEDDINGS
Dopo aver suddiviso il testo iniziale in tokens, è necessario convertire questi tokens in una rappresentazione numerica in modo che poi questa rappresentazione possa essere passata al modello per il training.
Il modello contiene l'embedding matrix che mappa i token creati col tokenizer ai rispettivi embeddings. Questo è il motivo per cui ogni modello deve utilizzare il rispettivo tokenizer.<br>
All'inizio del training questa matrice contiene valori random che vengono aggiornati nel corso del training


Verifico come utilizzare un modello transformer per generare dei contextualized token embeddings, cioè degli embeddings in cui il valore dipende dal contesto della frase.

In [17]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

In [18]:
output.shape

torch.Size([1, 4, 384])

In questo caso la frase Hello World è stata rappresentata da 4 tokens e ciascun token è stato rappresentato da un vettore di 384 VALORI

In [19]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
 world
[SEP]


L'output del language model è quindi il seguente

In [20]:
output

tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

# TEXT Embeddings

Dopo aver visto come un model può produrre token embeddings, è necessario capire come creare embeddings che rappresentino testi come frasi, paragrafi o interi documenti.<br>
Anche per questo scopo vengono utilizzati modelli di cui viene fatto un training per questo scopo specifico.<br>

Per utilizzare questi pretrained text embedding model si utilizza il package <b>SENTENCE-TRANSFORMERS</B>

In [23]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

vector.shape

(768,)

Si può quindi vedere come il testo in input sia stato convertito in un vettore di 768 interi che ne identificano il significato semantico